In [1]:
import os
import codecs
import re
import time
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [99]:
dataset_path = '../acts'
dataset = {}
TAG_RE = re.compile(r'<[^>]+>')
for file_path in os.listdir(dataset_path):
    with codecs.open(os.path.join(dataset_path, file_path), 'r', encoding='utf-8') as f:
        data = f.read()
        data = TAG_RE.sub('', data)
        data = " ".join(data.split())
        dataset[file_path] = data

In [38]:
for key, data in dataset.items():
    print(key, data[:3000])
    break

1.html ﻿ ЗАКОН о изгледу и употреби грба, заставе и химне Републике Србије “Службени гласник РС”, број 36 од 15. маја 2009. I. ОСНОВНЕ ОДРЕДБЕ Предмет закона Члан 1. Овим законом уређују се изглед и употреба грба, заставе и химне Републике Србије. Представљање Републике Србије Члан 2. Грбом, заставом и химном Републике Србије представља се Република Србија и изражава припадност Републици Србији. Грб, застава и химна Републике Србије могу се употребљавати само у облику и са садржином који су утврђени Уставом Републике Србије и овим законом. Основно правило о истицању грба и заставе Републике Србије Члан 3. Ако се грб, односно застава Републике Србије истиче у Републици Србији заједно са другим домаћим или страним грбовима или заставама, грб, односно застава Републике Србије ставља се на почасно место, ако овим законом није друкчије одређено. Ограничења приликом употребе грба и заставе Републике Србије Члан 4. На заставу и у грб Републике Србије не може се ништа исписати, односно уписати

In [100]:
def create_tfidf_features(corpus, max_features=5000):
    """ Creates a tf-idf matrix for the `corpus` using sklearn. """
    tfidf_vectorizor = TfidfVectorizer(decode_error='strict', strip_accents='utf-8', 
                                       analyzer='word', encoding='utf-8', norm='l2', 
                                       use_idf=True, smooth_idf=True, sublinear_tf=True,
                                       )
    X = tfidf_vectorizor.fit_transform(corpus)
    print('tfidf matrix successfully created.')
    return X, tfidf_vectorizor

In [111]:
print(type(list(dataset.values())[0]))
print(list(dataset.values())[9])

<class 'str'>
﻿ Zakon На основу члана 112. став 1. тачка 2. Устава Републике Србије, доносим УКАЗ о проглашењу Закона о потврђивању Споразума о зајму (Пројекат инклузивног предшколског образовања и васпитања) између Републике Србије и Међународне банке за обнову и развој Проглашава се Закон о потврђивању Споразума о зајму (Пројекат инклузивног предшколског образовања и васпитања) између Републике Србије и Међународне банке за обнову и развој, који је донела Народна скупштина Републике Србије, на Другој седници Другог редовног заседања, 8. новембра 2017. године. ПР број 54 У Београду, 10. новембра 2017. године Председник Републике, Александар Вучић, с.р. ЗАКОН о потврђивању Споразума о зајму (Пројекат инклузивног предшколског образовања и васпитања) између Републике Србије и Међународне банке за обнову и развој "Службени гласник РС - Међународни уговори", број 10 од 13. новембра 2017. Члан 1. Потврђује се Споразум о зајму (Пројекат инклузивног предшколског образовања и васпитања) између

In [90]:
corpus = list(dataset.values())
X, tf_idf_vectorizer = create_tfidf_features(corpus)

tfidf matrix successfully created.


In [91]:
X[0]


<1x133248 sparse matrix of type '<class 'numpy.float64'>'
	with 905 stored elements in Compressed Sparse Row format>

In [83]:
def calculate_similarity(X, vectorizor, query, top_k=10):
    """ Vectorizes the `query` via `vectorizor` and calculates the cosine similarity of
    the `query` and `X` (all the documents) and returns the `top_k` similar documents."""

    # Vectorize the query to the same length as documents
    query_vec = vectorizor.transform(query)
    # Compute the cosine similarity between query_vec and all the documents
    cosine_similarities = cosine_similarity(X,query_vec).flatten()
    # Sort the similar documents from the most similar to less similar and return the indices
    most_similar_doc_indices = np.argsort(cosine_similarities, axis=0)[:-top_k-1:-1]
    return (most_similar_doc_indices, cosine_similarities)

In [52]:
def show_similar_documents(df, cosine_similarities, similar_doc_indices):
    """ Prints the most similar documents using indices in the `similar_doc_indices` vector."""
    counter = 1
    for index in similar_doc_indices:
        print('Top-{}, Similarity = {}'.format(counter, cosine_similarities[index]))
        print('body: {}, '.format(df[index]))
        print()
        counter += 1

In [96]:
user_question = [u'педофилија']
search_start = time.time()
sim_vecs, cosine_similarities = calculate_similarity(X, tf_idf_vectorizer, user_question)
search_time = time.time() - search_start
print("search time: {:.2f} ms".format(search_time * 1000))
print()
show_similar_documents(list(dataset.keys()), cosine_similarities, sim_vecs)

search time: 15.00 ms

Top-1, Similarity = 0.04231735447776109
body: 83.html, 

Top-2, Similarity = 0.0
body: 999.html, 

Top-3, Similarity = 0.0
body: 249.html, 

Top-4, Similarity = 0.0
body: 243.html, 

Top-5, Similarity = 0.0
body: 244.html, 

Top-6, Similarity = 0.0
body: 245.html, 

Top-7, Similarity = 0.0
body: 246.html, 

Top-8, Similarity = 0.0
body: 247.html, 

Top-9, Similarity = 0.0
body: 248.html, 

Top-10, Similarity = 0.0
body: 25.html, 

